In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
groq_api_key: str = os.environ["GROQ_API_KEY"]

In [4]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph

In [5]:
graph = Neo4jGraph()

In [6]:
graph.query(
    """
MERGE (m:Movie {name:"Top Gun", runtime: 120})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

In [7]:
graph.refresh_schema()

In [8]:
print(graph.schema)

Node properties:
Movie {title: STRING, tagline: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT, runtime: INTEGER, name: STRING}
Person {born: INTEGER, name: STRING}
Database {name: STRING}
Message {name: STRING}
Product {unitPrice: FLOAT, productName: STRING, unitsInStock: INTEGER, productID: STRING, reorderLevel: INTEGER, supplierID: STRING, categoryID: STRING, discontinued: BOOLEAN, unitsOnOrder: INTEGER, quantityPerUnit: STRING}
Supplier {contactTitle: STRING, city: STRING, phone: STRING, address: STRING, companyName: STRING, supplierID: STRING, homePage: STRING, contactName: STRING, region: STRING, country: STRING, postalCode: STRING, fax: STRING}
Genre {name: STRING}
Actor {name: STRING}
Relationship properties:
ACTED_IN {roles: LIST}
REVIEWED {summary: STRING, rating: INTEGER}
The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:PRODUCED]->(:Movie)
(:Person)-[:WROTE]->(:Movie)
(:Person)-[:FOLLOWS]-

In [9]:
enhanced_graph = Neo4jGraph(
    enhanced_schema=True,
)
print(enhanced_graph.schema)

Node properties:
- **Movie**
  - `title`: STRING Example: "The Matrix"
  - `tagline`: STRING Example: "Welcome to the Real World"
  - `released`: INTEGER Min: 1964-12-16, Max: 2012
  - `id`: STRING Example: "1"
  - `imdbRating`: FLOAT Min: 2.4, Max: 9.3
  - `runtime`: INTEGER Min: 120, Max: 120
  - `name`: STRING Available options: ['Top Gun']
- **Person**
  - `born`: INTEGER Min: 1929, Max: 1996
  - `name`: STRING Example: "Keanu Reeves"
- **Database**
  - `name`: STRING Available options: ['Neo4j']
- **Message**
  - `name`: STRING Available options: ['Hello World!']
- **Product**
  - `unitPrice`: FLOAT Min: 2.5, Max: 263.5
  - `productName`: STRING Example: "Chai"
  - `unitsInStock`: INTEGER Min: 0, Max: 125
  - `productID`: STRING Example: "1"
  - `reorderLevel`: INTEGER Min: 0, Max: 30
  - `supplierID`: STRING Example: "1"
  - `categoryID`: STRING Available options: ['1', '2', '7', '6', '8', '4', '3', '5']
  - `discontinued`: BOOLEAN 
  - `unitsOnOrder`: INTEGER Min: 0, Max: 100
  

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

In [11]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True
)

In [12]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'I don\'t have information on that topic. However, I can tell you that the main cast of the 1986 film "Top Gun" included Tom Cruise, Kelly McGillis, Val Kilmer, and Anthony Edwards.'}

In [13]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, top_k=2
)
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'I don\'t have information on that topic. However, I can tell you that the main cast of the 1986 film "Top Gun" included Tom Cruise, Kelly McGillis, Val Kilmer, and Anthony Edwards.'}

In [14]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, return_intermediate_steps=True
)
result = chain.invoke({"query": "Who played in Top Gun?"})
print(f"Intermediate steps: {result['intermediate_steps']}")
print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[]

> Finished chain.
Intermediate steps: [{'query': "MATCH (m:Movie)-[:ACTED_IN]->(p:Person)\nWHERE m.title = 'Top Gun'\nRETURN p.name"}, {'context': []}]
Final answer: I don't have information on that topic. However, I can tell you that the main cast of the 1986 film "Top Gun" included Tom Cruise, Kelly McGillis, Val Kilmer, and Anthony Edwards.


In [15]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, return_direct=True
)
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name

> Finished chain.


{'query': 'Who played in Top Gun?', 'result': []}

In [16]:
from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [17]:
chain.invoke({"query": "How many people played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title:"Top Gun"})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors
Full Context:
[{'numberOfActors': 6}]

> Finished chain.


{'query': 'How many people played in Top Gun?',
 'result': 'Six people played in Top Gun.'}

In [18]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=llm,
    qa_llm=llm,
    verbose=True,
)

In [19]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'I don\'t have information on that topic. However, I can tell you that the main cast of the 1986 film "Top Gun" included Tom Cruise, Kelly McGillis, Val Kilmer, and Anthony Edwards.'}

In [20]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=llm,
    qa_llm=llm,
    verbose=True,
    exclude_types=["Movie"],
)
# Inspect graph schema
print(chain.graph_schema)

Node properties are the following:
Person {born: INTEGER, name: STRING},Database {name: STRING},Message {name: STRING},Product {unitPrice: FLOAT, productName: STRING, unitsInStock: INTEGER, productID: STRING, reorderLevel: INTEGER, supplierID: STRING, categoryID: STRING, discontinued: BOOLEAN, unitsOnOrder: INTEGER, quantityPerUnit: STRING},Supplier {contactTitle: STRING, city: STRING, phone: STRING, address: STRING, companyName: STRING, supplierID: STRING, homePage: STRING, contactName: STRING, region: STRING, country: STRING, postalCode: STRING, fax: STRING},Genre {name: STRING},Actor {name: STRING}
Relationship properties are the following:
ACTED_IN {roles: LIST},REVIEWED {summary: STRING, rating: INTEGER}
The relationships are the following:
(:Person)-[:FOLLOWS]->(:Person),(:Database)-[:SAYS]->(:Message)


In [21]:
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    validate_cypher=True,
)
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[{'p.name': 'Val Kilmer'}, {'p.name': 'Meg Ryan'}, {'p.name': 'Tom Skerritt'}, {'p.name': 'Kelly McGillis'}, {'p.name': 'Tom Cruise'}, {'p.name': 'Anthony Edwards'}]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'Val Kilmer, Meg Ryan, Tom Skerritt, Kelly McGillis, Tom Cruise, and Anthony Edwards played in Top Gun.'}